This notebook is ran in a docker container where the project directory (i.e. same directory as README.md) is located in `/code`, which is set below. If you run locally you'll need to set the path of your project directory accordingly.


In [1]:
%cd /code

/code


---

The `load_dotenv` function below loads all the variables found in the `.env` file as environment variables. You must have a `.env` file located in the project directory containing your OpenAI API key, in the following format.

```
OPENAI_API_KEY=sk-...
```

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

---

## Memory

The default behavior of `OpenAIChat` is to add each prompt/response to/from the model to the history and use that history as the context for future prompts sent to the model (for that object's lifetime). For short conversations, this is probably ideal. For longer conversations, the number of tokens can dramatically increase, which can increase costs, and will eventually grow larger than the maximum context length allowed.

In [3]:
from llm_workflow.openai import OpenAIChat

chat = OpenAIChat()
response = chat("Hi, my name is Shane. My favorite number is 42 and my favorite color is blue.")
response

"Nice to meet you, Shane! 42 is a great number, and blue is a lovely color. Is there anything specific you'd like to know or talk about?"

The `_previous_memory` property of the `OpenAIChat` object shows the list of messages sent to the chat model for the prior interaction.

In [4]:
# _previous_messages stores the list of messages sent to the OpenAI model.
chat._previous_messages

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user',
  'content': 'Hi, my name is Shane. My favorite number is 42 and my favorite color is blue.'}]

In [5]:
response = chat("What is my favorite number?")
response

'Your favorite number is 42.'

In [6]:
chat._previous_messages

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user',
  'content': 'Hi, my name is Shane. My favorite number is 42 and my favorite color is blue.'},
 {'role': 'assistant',
  'content': "Nice to meet you, Shane! 42 is a great number, and blue is a lovely color. Is there anything specific you'd like to know or talk about?"},
 {'role': 'user', 'content': 'What is my favorite number?'}]

In [7]:
response = chat("What is my favorite color?")
response

'Your favorite color is blue.'

In [8]:
chat._previous_messages

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user',
  'content': 'Hi, my name is Shane. My favorite number is 42 and my favorite color is blue.'},
 {'role': 'assistant',
  'content': "Nice to meet you, Shane! 42 is a great number, and blue is a lovely color. Is there anything specific you'd like to know or talk about?"},
 {'role': 'user', 'content': 'What is my favorite number?'},
 {'role': 'assistant', 'content': 'Your favorite number is 42.'},
 {'role': 'user', 'content': 'What is my favorite color?'}]

---

## LastNExchangesManager


The `OpenAIChat` model has a `memory_manager` parameter and takes a `MemoryManager` class. A `MemoryManager` class is a callable that takes a `list[ExchangeRecord]` (i.e. from the `model.history()` property) and also returns a `list[ExchangeRecord]` serving as the model's memory (i.e. a list containing the messages that will be sent to the model along with the new prompt). This allows the end user to easily define a memory strategy of their own (e.g. keep the first message and the last `n` messages).

One Example of a `MemoryManager` is a `LastNExchangesManager` class where you can specify the last `n` messages that you want to keep.

In [9]:
from llm_workflow.openai import OpenAIChat
from llm_workflow.memory import LastNExchangesManager

# only remember the last message (i.e. prompt/response)
chat = OpenAIChat(memory_manager=LastNExchangesManager(last_n_exchanges=1))
response = chat("Hi, my name is Shane. My favorite number is 42 and my favorite color is blue.")
response

"Nice to meet you, Shane! 42 is a great number, and blue is a lovely color. Is there anything specific you'd like to talk about or ask?"

In [10]:
chat._previous_messages

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user',
  'content': 'Hi, my name is Shane. My favorite number is 42 and my favorite color is blue.'}]

In [11]:
response = chat("What is my favorite number?")
response

'Your favorite number is 42.'

In [12]:
chat._previous_messages

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user',
  'content': 'Hi, my name is Shane. My favorite number is 42 and my favorite color is blue.'},
 {'role': 'assistant',
  'content': "Nice to meet you, Shane! 42 is a great number, and blue is a lovely color. Is there anything specific you'd like to talk about or ask?"},
 {'role': 'user', 'content': 'What is my favorite number?'}]

Since we are only keeping the last message (i.e. the last prompt/response), the model will not have access to my name or favorite color.

In [13]:
response = chat("What is my favorite color?")
response

"I'm sorry, I don't have that information. Can you please tell me your favorite color?"

In [14]:
chat._previous_messages

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user', 'content': 'What is my favorite number?'},
 {'role': 'assistant', 'content': 'Your favorite number is 42.'},
 {'role': 'user', 'content': 'What is my favorite color?'}]

---